In [0]:
import matplotlib
!pip install imutils
!pip install opencv-python
!pip install q keras==2.3.0
!pip install tensorflow_hub

In [0]:
import keras
keras.__version__

'2.3.0'

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import h5py
import matplotlib
import pickle
import tensorflow as tf
import tensorflow_hub as hub
from random import shuffle
import glob
import numpy as np
import h5py
import cv2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
hdf5_path = '/content/drive/My Drive/VideoTestCases/VideoTestCases224V5.hdf5'
dataset = h5py.File(hdf5_path, "r")
print(dataset["train_labels"].shape)
train_labels=np.array(dataset["train_labels"])
unique_lables_set = set(train_labels) 

train_labels = train_labels.reshape((len(train_labels),-1))
train_batch_labels=train_labels

unique_labels_list = (list(unique_lables_set))

print(len(unique_labels_list))
print(unique_labels_list)

(3615,)
3
[0, 1, 2]


In [0]:
print(dataset["train_labels"].shape)

(3615,)


In [0]:
train_img=np.array(dataset["train_img"])
#train_img = train_img.reshape((len(train_img),-1))
print(len(train_img))
train_img[0].shape

3615


(224, 224, 3)

In [0]:
train_batch_imgs=[]
for i in range(len(train_batch_labels)):
   img=(dataset['train_img'])[i]
   img=img/255.
   train_batch_imgs.append(img)    
train_batch_imgs=np.array(train_batch_imgs)
train_batch_imgs[0].shape

(224, 224, 3)

In [0]:
INIT_LR = 1e-1
BS = 15
EPOCHS = 50

data = train_batch_imgs
labels = train_batch_labels

In [0]:
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels, 3)
with open('/content/drive/My Drive/3kImages/label_encoder_file_0402.pkl', 'wb') as fid:
    pickle.dump(le, fid)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
trainX[0].shape

(224, 224, 3)

In [0]:
print("[INFO] performing 'on the fly' data augmentation")
aug = ImageDataGenerator(
    rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

[INFO] performing 'on the fly' data augmentation


In [0]:
print("[INFO] compiling model...")
model = tf.keras.Sequential([ hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                              output_shape=[1280], trainable=False), tf.keras.layers.Dropout(0.4),
                              tf.keras.layers.Dense(len(unique_labels_list), activation='softmax')])
model.build([None, 224, 224, 3])
model.summary()

model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss='categorical_crossentropy',
      metrics=['acc'])

[INFO] compiling model...
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_7 (KerasLayer)   multiple                  2257984   
_________________________________________________________________
dropout_7 (Dropout)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

print(H)

model.save("/content/drive/My Drive/3kImages/models/ImageQualityV11.h5")
model.save_weights("/content/drive/My Drive/3kImages/models/ImageQualityV11Weights.h5")

[INFO] training network for 50 epochs...
Epoch 1/50
180/180 [==============================] - 38s 209ms/step - loss: 1.3547 - acc: 0.4026 - val_loss: 1.1262 - val_acc: 0.4679
Epoch 2/50
180/180 [==============================] - 36s 202ms/step - loss: 1.2889 - acc: 0.4169 - val_loss: 1.2151 - val_acc: 0.3319
Epoch 3/50
180/180 [==============================] - 36s 202ms/step - loss: 1.2201 - acc: 0.4169 - val_loss: 1.1129 - val_acc: 0.3883
Epoch 4/50
180/180 [==============================] - 36s 200ms/step - loss: 1.1818 - acc: 0.4243 - val_loss: 1.0658 - val_acc: 0.4701
Epoch 5/50
180/180 [==============================] - 36s 198ms/step - loss: 1.1393 - acc: 0.4336 - val_loss: 1.1178 - val_acc: 0.3750
Epoch 6/50
180/180 [==============================] - 37s 207ms/step - loss: 1.1427 - acc: 0.4329 - val_loss: 1.1104 - val_acc: 0.3507
Epoch 7/50
180/180 [==============================] - 36s 199ms/step - loss: 1.1399 - acc: 0.4321 - val_loss: 1.0619 - val_acc: 0.4425
Epoch 8/50
180

KeyboardInterrupt: ignored

In [0]:
from keras import models
#new_model = models.load_model('/content/drive/My Drive/3kImages/models/ImageQualityX1.h5')
new_model = tf.keras.models.load_model('/content/drive/My Drive/3kImages/models/ImageQualityX1.h5',custom_objects={'KerasLayer':hub.KerasLayer})
new_model.summary()

In [0]:
test_image = image.load_img('/content/drive/My Drive/VideoTestCases/test/Normal/TAJframe3427.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255

print(test_image.shape)

result = new_model.predict(test_image)
print(result)
#predict_proba = new_model.predict_proba(test_image)
prediction_result = np.argmax(result[0])
print(prediction_result)

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os

# image folder
folder_path = '/content/drive/My Drive/3kImages/miniTest/MacroBlocking'
# path to model
model_path = '/path/to/saved/model.h5'
# dimensions of images
img_width, img_height = 224, 224


# load all images into a list
images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)
    #print(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = new_model.predict_classes(images, batch_size=10)
print(classes)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

data_root='/content/drive/My Drive/VideoTestCases/test'

IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(directory=TRAINING_DATA_DIR, target_size = (224, 224),
                                            batch_size = 876, shuffle=False)

sample_training_images, _ = next(test_set)
print(test_set.labels[800:815])

plt.figure(figsize=(12, 12))

def plotImages(images_arr):
    fig, axes = plt.subplots(1, 20, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
# plotImages(sample_training_images[800:815])

for i in range(40):
  plt.figure(figsize=(12, 12))
  #plt.imshow(sample_training_images[i+800])
  #plt.show()

total_validate = 44
batch_size = 32


#predict = new_model.predict(test_set, steps=np.ceil(total_validate/32))

print('before running prediction')
predict = new_model.predict(sample_training_images)
print('after running prediction')
predict=np.round(predict,decimals=5)
result = map(lambda v : np.argmax(v), predict)
prediction_result = np.array(list(result))
print(prediction_result[800:815])
#print(predict)

#test_labels = np.array(np.zeros(44))
#print(test_labels.shape)
#my_data = (test_set, test_labels)
#test_loss, test_acc = new_model.evaluate(my_data)
#print(test_acc)


In [0]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

classification_report = metrics.classification_report(test_set.labels, prediction_result, target_names = test_set.class_indices.keys())
print("Classification report: \n", classification_report)
confusion_matrix = metrics.confusion_matrix(test_set.labels, prediction_result)
print("Confusion matrix: \n",confusion_matrix)

print("Accuracy Score: ", accuracy_score(test_set.labels, prediction_result))
print("F1 Score: ", f1_score(test_set.labels, prediction_result, average='macro'))
print("F1 Score: ", f1_score(test_set.labels, prediction_result, average='micro'))
print("F1 Score: ", f1_score(test_set.labels, prediction_result, average='weighted'))

In [0]:
import cv2
from keras.preprocessing import image
vidcap = cv2.VideoCapture('/content/drive/My Drive/3kImages/faces_noblur.mp4')
print(cv2.__version__)

success,img = vidcap.read()
count = 0
newsize = (224, 224) 
images = []
while success:
  img = cv2.resize(img, newsize, interpolation = cv2.INTER_AREA)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)

  prediction_probability = new_model.predict(img)
  prediction_result = np.argmax(prediction_probability[0])
  images.append(prediction_result)
  # cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file

  success,img = vidcap.read()
  count += 1

print('final result')
print(images)
vidcap.release()
cv2.destroyAllWindows()

In [0]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

class ResNet:
    @staticmethod
    def residual_module(data, K, stride, chanDim, red=False,
        reg=0.0001, bnEps=2e-5, bnMom=0.9):
        # the shortcut branch of the ResNet module should be
        # initialize as the input (identity) data
        shortcut = data

        # the first block of the ResNet module are the 1x1 CONVs
        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(data)
        act1 = Activation("relu")(bn1)
        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
            kernel_regularizer=l2(reg))(act1)

        # the second block of the ResNet module are the 3x3 CONVs
        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(conv1)
        act2 = Activation("relu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
            padding="same", use_bias=False,
            kernel_regularizer=l2(reg))(act2)

        # the third block of the ResNet module is another set of 1x1
        # CONVs
        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(conv2)
        act3 = Activation("relu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False,
            kernel_regularizer=l2(reg))(act3)

        # if we are to reduce the spatial size, apply a CONV layer to
        # the shortcut
        if red:
            shortcut = Conv2D(K, (1, 1), strides=stride,
                use_bias=False, kernel_regularizer=l2(reg))(act1)

        # add together the shortcut and the final CONV
        x = add([conv3, shortcut])

        # return the addition as the output of the ResNet module
        return x

    @staticmethod
    def build(width, height, depth, classes, stages, filters,
        reg=0.0001, bnEps=2e-5, bnMom=0.9):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # set the input and apply BN
        inputs = Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(inputs)

        # apply CONV => BN => ACT => POOL to reduce spatial size
        x = Conv2D(filters[0], (5, 5), use_bias=False,
            padding="same", kernel_regularizer=l2(reg))(x)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(x)
        x = Activation("relu")(x)
        x = ZeroPadding2D((1, 1))(x)
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)

        # loop over the number of stages
        for i in range(0, len(stages)):
            # initialize the stride, then apply a residual module
            # used to reduce the spatial size of the input volume
            stride = (1, 1) if i == 0 else (2, 2)
            x = ResNet.residual_module(x, filters[i + 1], stride,
                chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

            # loop over the number of layers in the stage
            for j in range(0, stages[i] - 1):
                # apply a ResNet module
                x = ResNet.residual_module(x, filters[i + 1],
                    (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

        # apply BN => ACT => POOL
        x = BatchNormalization(axis=chanDim, epsilon=bnEps,
            momentum=bnMom)(x)
        x = Activation("relu")(x)
        x = AveragePooling2D((8, 8))(x)

        # softmax classifier
        x = Flatten()(x)
        x = Dense(classes, kernel_regularizer=l2(reg))(x)
        x = Activation("softmax")(x)

        # create the model
        model = Model(inputs, x, name="resnet")

        # return the constructed network architecture
        return model

In [0]:
'''
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / EPOCHS)
model = ResNet.build(144, 256, 3, 3, (2, 3, 4),
    (32, 128, 128, 256), reg=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
'''

[INFO] compiling model...
